In [1]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
import statistics as stat
from scipy import stats
lone_pine = pd.read_csv(r'C:\Users\wells\OneDrive\Desktop\lone_pine.csv')
crsp_data = pd.read_csv('StockPriceMonthly.csv', encoding='utf8', low_memory=False)

# pre-processing

In [2]:
lone_pine.drop(['% Port', '% OS', 'Hist', 'Change', 'Change.1'], axis=1, inplace=True)

In [3]:
lone_pine['Ticker'] = lone_pine['Ticker'].astype('str')
lone_pine['Class'] = lone_pine['Class'].astype('str')
def ab(aa):
    if '.' in aa:
        return aa.split('.')[0]
    if '/' in aa:
        return aa.split('/')[0]
    else:
        return aa
lone_pine['Ticker'] = lone_pine['Ticker'].apply(ab)

In [4]:
def abc(aa):
    aa = aa.split('/')
    year = aa[2]
    month = aa[0]
    day = aa[1]
    return year+month
lone_pine['Date'] = lone_pine['Date'].apply(abc)

In [5]:
lone_pine = lone_pine[lone_pine['Class']!='PUT'].loc[lone_pine['Class']!='CALL']
lone_pine = lone_pine[lone_pine['Class']!='put'].loc[lone_pine['Class']!='call']
lone_pine = lone_pine[lone_pine['Class']!='Put'].loc[lone_pine['Class']!='Call']

In [6]:
def class_3str(df_col):
    df_col = df_col.replace(' ', '')
    df_col = df_col[0:3]
    df_col = df_col.upper()
    return df_col
lone_pine['Class'] = lone_pine['Class'].apply(class_3str)

In [7]:
lone_pine_not = lone_pine[lone_pine['Ticker']!='nan']
lone_pine_is = lone_pine[lone_pine['Ticker']=='nan']

In [8]:
lone_pine_not.drop('Name', axis=1, inplace=True)
lone_pine_not = lone_pine_not.groupby(['Ticker','Class','Date'])[['Shares', 'Value']].agg('sum').reset_index().copy()

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
lone_pine_is.loc[:, 'Ticker'] = lone_pine_is['Name']
lone_pine_is.drop('Name', axis=1, inplace=True)
lone_pine_is = lone_pine_is.groupby(['Ticker','Class','Date'])[['Shares', 'Value']].agg('sum').reset_index().copy()

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
date = sorted(list(set(lone_pine_not['Date'])))

In [11]:
new_lone_pine = pd.DataFrame()

In [12]:
for num in range(len(date)-1):
    first_date = date[num]
    second_date = date[num+1]
    first_df = lone_pine_not[lone_pine_not['Date']==first_date]
    second_df = lone_pine_not[lone_pine_not['Date']==second_date]
    first_df = first_df.drop(['Date'],axis=1)
    first_df.columns = ['Ticker', 'Class', 'Last_Shares', 'Last_Value']
    merge_df = second_df.merge(first_df, on=['Ticker','Class'], how='left')
    if len(merge_df) != len(second_df):
        print(second_date)
    new_lone_pine = pd.concat([new_lone_pine, merge_df], ignore_index=True)

In [13]:
new_lone_pine_is = pd.DataFrame()

In [14]:
for num in range(len(date)-1):
    first_date = date[num]
    second_date = date[num+1]
    first_df = lone_pine_is[lone_pine_is['Date']==first_date]
    second_df = lone_pine_is[lone_pine_is['Date']==second_date]
    first_df = first_df.drop(['Date'],axis=1)
    first_df.columns = ['Ticker', 'Class', 'Last_Shares', 'Last_Value']
    merge_df = second_df.merge(first_df, on=['Ticker','Class'], how='left')
    if len(merge_df) != len(second_df):
        print(second_date)
    new_lone_pine_is = pd.concat([new_lone_pine_is, merge_df], ignore_index=True, sort=False)

In [15]:
new_lone_pine = pd.concat([new_lone_pine, lone_pine_not[lone_pine_not['Date']==date[0]]], ignore_index=True, sort=False)
new_lone_pine_is = pd.concat([new_lone_pine_is, lone_pine_is[lone_pine_is['Date']==date[0]]],ignore_index=True, sort=False)
lone_pine_a = pd.concat([new_lone_pine, new_lone_pine_is], ignore_index=True, sort=False)

In [16]:
lone_pine_a = lone_pine_a.replace(np.nan, 0)

In [17]:
lone_pine_a.loc[:, 'Shares_Change'] = lone_pine_a['Shares'] - lone_pine_a['Last_Shares']
lone_pine_a.loc[:, 'Value_Change'] = lone_pine_a['Value'] - lone_pine_a['Last_Value']

In [18]:
lone_pine_a.drop(['Last_Shares', 'Last_Value'], axis=1, inplace=True)

In [19]:
add_df = pd.DataFrame(columns=lone_pine_a.columns)
for index in tqdm_notebook(range(len(lone_pine_a))):
    each_data = lone_pine_a.loc[index]
    each_data = list(each_data)
    date = int(each_data[2])
    if date % 4 == 0:
        date = date - 12 + 100
    each_data[2] = str(date+1)
    each_data[5] = 0
    each_data[6] = 0
    add_df.loc[-1] = each_data
    add_df = add_df.reset_index(drop=True)
    date = int(each_data[2])
    each_data[2] = str(date+1)
    each_data[5] = 0
    each_data[6] = 0
    add_df.loc[-1] = each_data
    add_df = add_df.reset_index(drop=True)
lone_pine_a = pd.concat([lone_pine_a, add_df], ignore_index=True, sort=False)

In [20]:
lone_pine_a.drop(lone_pine_a[lone_pine_a['Shares']==0].loc[lone_pine_a['Value']==0].loc[lone_pine_a['Shares_Change']==0].loc[lone_pine_a['Value_Change']==0].index, inplace=True)

In [21]:
def class_(df_col):
    if df_col == 'CLA':
        return 'A'
    elif df_col == 'CLB':
        return 'B'
    elif df_col == 'CLC':
        return 'C'
    elif df_col == 'CLD':
        return 'D'
    else:
        return df_col
lone_pine_a['Class'] = lone_pine_a['Class'].apply(class_)

In [22]:
def division(df):
    if df['Shares_Change'] == 0 and df['Shares'] == 0:
        return 0
    elif df['Shares'] == 0 and df['Value'] == 0:
        return df['Value_Change'] / df['Shares_Change']
    else:
        return df['Value'] / df['Shares']
lone_pine_a['price'] = lone_pine_a.apply(division, axis=1)

In [23]:
def abcd(aa):
    return str(aa)[0:6]
crsp_data['date'] = crsp_data['date'].apply(abcd)

In [24]:
crsp_data.columns = ['PERMNO', 'Date', 'Ticker', 'COMNAM', 'Class', 'DIVAMT', 'PRC', 'VOL']

In [25]:
def to_abs(df_col):
    return abs(df_col)
crsp_data_c = crsp_data[['Date', 'Ticker', 'PRC', 'Class']]
crsp_data_c = crsp_data_c[crsp_data_c['Ticker'].isin(list(set(lone_pine_a['Ticker'])))]
crsp_data_c = crsp_data_c[crsp_data_c['PRC'].notnull()].drop_duplicates(keep='first')
crsp_data_c['PRC'] = crsp_data_c['PRC'].apply(to_abs)

In [26]:
# crsp_data_c = crsp_data_c.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
# df_list = []
# for each in tqdm_notebook(crsp_data_c.groupby(by='Ticker')):
#     ticker, df = each
#     df.loc[:, 'Last_PRC'] = df['PRC'].shift()
#     df_list.append(df)

In [27]:
import multiprocessing as mp
import time
from function import test_function
if __name__ == '__main__':
    a = time.time()
    pool_list = [i for i in crsp_data_c.groupby(by='Ticker')]
    pool = mp.Pool(processes=5)
    df_list = pool.map(test_function.last_price, pool_list)
    b = time.time()
    print(b - a)

1.8494412899017334


In [28]:
crsp_data_c = pd.concat(df_list)

In [29]:
lone_pine_all = lone_pine_a.merge(crsp_data_c, on=['Date', 'Ticker'], how = 'left')

In [30]:
for ticker in list(set(list(lone_pine_all['Ticker']))):
    if len(lone_pine_a[lone_pine_a['Ticker']==ticker].reset_index(drop=True)) != len(lone_pine_all[lone_pine_all['Ticker']==ticker].reset_index(drop=True)):
        drop_df = lone_pine_all[lone_pine_all['Ticker']==ticker].loc[lone_pine_all['Class_x']!=lone_pine_all['Class_y']]
        lone_pine_all.drop(list(drop_df.index), inplace=True)

In [31]:
# lone_pine_all = lone_pine_all.sort_values(by='Date').reset_index(drop=True)
# df_list = []
# for each in tqdm_notebook(lone_pine_all.groupby(by='Ticker')):
#     ticker, df = each
#     df.loc[:, 'Last_price'] = df['price'].shift()
#     df.loc[:, 'Last_PRC'] = df['PRC'].shift()
#     df_list.append(df)

In [32]:
# lone_pine_all = pd.concat(df_list)

In [33]:
error = lone_pine_all[lone_pine_all['PRC'].isnull()]

In [34]:
test = lone_pine_all
new_error = []
for index in tqdm_notebook(error.index):
    each = error.loc[index]
    df = crsp_data[crsp_data['Ticker']==each['Ticker']]
    try:
        if int(each['Date']) > int(list(df['Date'])[-1]):
            test = test.drop(index)
        else:
            new_error.append(list(each))
    except:
        new_error.append(list(each))
lone_pine_all = test

In [35]:
new_error = pd.DataFrame(new_error)
new_error.columns = lone_pine_all.columns
new_error = new_error.sort_values(by='Date').reset_index(drop=True)

In [36]:
df_list = []
for each in tqdm_notebook(new_error.groupby(by='Ticker')):
    ticker, df = each
    df.loc[:, 'PRC'] = df['price']
    df.loc[:, 'Last_PRC'] = df['PRC'].shift()
    df_list.append(df)
new_error = pd.concat(df_list)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [37]:
lone_pine_all = pd.concat([lone_pine_all[lone_pine_all['PRC'].notnull()], new_error], ignore_index=True, sort=False)

In [38]:
pd.DataFrame(new_error).to_csv('error.csv', index=False)

In [39]:
lone_pine_all = lone_pine_all.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

In [40]:
test = lone_pine_all
for each in lone_pine_all.groupby('Ticker'):
    ticker, df = each
    for index in df.index[:-1]:
        if df.loc[index]['Shares']==0 and df.loc[index+1]['Shares']==0:
            test.drop(index+1, inplace=True)

# revenue

In [41]:
lone_pine_all.drop(['Class_x', 'price'], axis=1, inplace=True)
lone_pine_all.columns = ['Ticker', 'Date', 'Shares', 'Value', 'Shares_Change', 'Value_Change', 'Price',
                         'Class', 'Last_Price']

In [42]:
new_share = lone_pine_all.loc[lone_pine_all['Shares']==lone_pine_all['Shares_Change']]
new_share.loc[:,'New_Value'] = new_share['Shares'] * new_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [43]:
new_share.to_csv('new_share.csv', index=False)

In [44]:
sell_share = lone_pine_all[lone_pine_all['Shares']==0]
sell_share.loc[:,'New_Value'] = sell_share['Shares_Change'] * sell_share['Price']

In [45]:
sell_share.to_csv('sell_share.csv', index=False)

In [46]:
more_share = lone_pine_all[lone_pine_all['Shares_Change']>0].loc[lone_pine_all['Shares']!=lone_pine_all['Shares_Change']]
more_share.loc[:,'New_Value'] = more_share['Shares'] * more_share['Price']

In [47]:
more_share.to_csv('more_share.csv', index=False)

In [48]:
less_share = lone_pine_all[lone_pine_all['Shares_Change']<0].loc[lone_pine_all['Shares']!=0]
less_share.loc[:,'New_Value'] = (less_share['Shares'] + abs(less_share['Shares_Change'])) * less_share['Price']

In [49]:
less_share.to_csv('less_share.csv', index=False)

In [50]:
same_share = lone_pine_all[lone_pine_all['Shares_Change']==0]
same_share.loc[:,'New_Value'] = same_share['Shares'] * same_share['Price']

In [51]:
same_share.to_csv('same_share.csv', index=False)

In [52]:
lone_pine_all.loc[:,'New_Value'] = lone_pine_all['Shares'] * lone_pine_all['Price']

In [53]:
total_asset = []
for date in list(set(lone_pine_all['Date'])):
    lone_pine_all_t = lone_pine_all[lone_pine_all['Date']==date]
    total_asset.append(sum(lone_pine_all_t['New_Value']))

In [54]:
revenue = []
for date in list(set(lone_pine_all['Date'])):
    lone_pine_all_t = lone_pine_all[lone_pine_all['Date']==date]

    sell_shares = lone_pine_all_t[lone_pine_all_t['Shares']==0]
    more_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']>0].loc[lone_pine_all_t['Shares']!=lone_pine_all_t['Shares_Change']]
    less_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']<0].loc[lone_pine_all_t['Shares']!=0]
    same_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']==0]

    sell_shares.loc[:,'diff'] = abs(sell_shares['Shares_Change']) * ((sell_shares['Price'] - sell_shares['Last_Price']) / 2)
    more_shares.loc[:,'diff'] = (more_shares['Shares'] - more_shares['Shares_Change']) * (more_shares['Price'] - more_shares['Last_Price'])
    less_shares.loc[:,'diff'] = less_shares['Shares'] * (less_shares['Price'] - less_shares['Last_Price']) + abs(less_shares['Shares_Change']) * ((less_shares['Price'] - less_shares['Last_Price']) / 2)
    same_shares.loc[:,'diff'] = same_shares['Shares'] * (same_shares['Price'] - same_shares['Last_Price'])
    
    sell_shares = sell_shares[sell_shares['diff'].notnull()]
    more_shares = more_shares[more_shares['diff'].notnull()]
    less_shares = less_shares[less_shares['diff'].notnull()]
    same_shares = same_shares[same_shares['diff'].notnull()]
    
    revenue.append(sum(sell_shares['diff']) + sum(more_shares['diff']) + sum(less_shares['diff']) + sum(same_shares['diff']))

In [55]:
revenue_df = pd.DataFrame({'date':list(set(lone_pine_all['Date'])),
                          'asset':total_asset,
                          'revenue':revenue})

In [56]:
revenue_df = revenue_df.sort_values(by='date').reset_index(drop=True)

In [57]:
revenue_df['last_asset'] = revenue_df['asset'].shift()

In [58]:
revenue_df['return_rate'] = round(revenue_df['revenue'] / revenue_df['last_asset'] * 100, 4)

In [59]:
revenue_df = revenue_df[0:-2].reset_index(drop=True)

In [60]:
revenue_df.to_csv('revenue.csv', index=False)

In [61]:
revenue_df = revenue_df[1:]
revenue_df = revenue_df[revenue_df['return_rate']<40].loc[revenue_df['return_rate']>(-40)]

In [62]:
fama = pd.read_csv('fama.csv')
fama['date'] = fama['date'].astype('str')

fama_3 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_3 = fama_3[fama_3['return_rate'].notnull()]

fama_3['return_rate'] = fama_3['return_rate'] - fama_3['RF']

In [63]:
fama = pd.read_csv('fama5.csv')
fama['date'] = fama['date'].astype('str')

fama_5 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_5 = fama_5[fama_5['return_rate'].notnull()]

fama_5['return_rate'] = fama_5['return_rate'] - fama_5['RF']

# lasso

In [81]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import f_regression

In [99]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [105]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
# alpha = 10
# while alpha > 0.01:
for i in range(10):
    x = fama_3[['Mkt-RF', 'SMB', 'HML']].values
    y = fama_3[['return_rate']].values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

    sc = StandardScaler()
    sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
    x_train_std = sc.transform(x_train)
    x_test_std = sc.transform(x_test)
    lassocv = LassoCV()
    lassocv.fit(x_train_std, y_train)
    alpha = lassocv.alpha_
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train)
    y_predict_train = lasso.predict(x_train_std)
    score_train = r2_score(y_train, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score = r2_score(y_test, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    e = mean_squared_error(y_test, y_predict)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
#     alpha = alpha - 0.01

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selectio

In [106]:
alpha_df.sort_values(by='test_score', ascending=False)

,alpha,coef1,coef2,coef3,train_score,test_score,e,coeff_used
4,0.003553,3.881883,0.155247,-1.319709,0.579118,0.873415,3.854697,3.0
7,0.034140,4.241306,0.000000,-1.385199,0.624422,0.836024,3.481612,2.0
1,0.104560,4.041007,-0.000000,-1.001369,0.620623,0.819369,4.311621,2.0
2,0.041759,4.236401,0.000000,-1.489033,0.626207,0.811061,3.679202,2.0
8,0.088802,4.089489,0.000000,-1.435677,0.632959,0.783468,5.166364,2.0
5,0.004082,4.273964,0.264646,-1.266133,0.643415,0.761484,4.608148,3.0
9,0.076253,4.344211,0.000000,-1.252608,0.651617,0.723442,5.612384,2.0
0,0.108908,4.299477,0.000000,-1.342961,0.658074,0.696805,5.582236,2.0
6,0.070800,4.491572,0.007387,-1.202729,0.670821,0.628125,4.709472,3.0
3,0.053030,4.519074,0.016663,-1.248883,0.674056,0.620129,5.531237,3.0


In [325]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [341]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'coef1_p', 'coef2_p', 'coef3_p', 'coef4_p', 'coef5_p', 'train_score', 'test_score', 'e'])
alpha = 3
while alpha > 0.1:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train=r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    p_value = list(f_regression(x_test.values, y_predict)[1])
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + p_value + [score_train, score, e]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.1

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-pa

In [337]:
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

train_score: 0.610228573517401
test_score: 0.8464889587156124


,alpha,coef1,coef2,coef3,coef4,coef5,coef1_p,coef2_p,coef3_p,coef4_p,coef5_p,train_score,test_score,e
28,0.2,3.476344,0.0,0.0,-0.824973,-0.619526,8.748954e-17,0.000318,0.00504,0.073089,0.777133,0.610229,0.846489,4.315371
27,0.3,3.357322,0.0,0.0,-0.786695,-0.512318,8.748954e-17,0.000318,0.00504,0.073089,0.777133,0.604931,0.843261,4.406103
26,0.4,3.238300,0.0,0.0,-0.748417,-0.405110,8.748954e-17,0.000318,0.00504,0.073089,0.777133,0.597514,0.837466,4.569024
25,0.5,3.119226,0.0,0.0,-0.710211,-0.297857,8.748954e-17,0.000318,0.00504,0.073089,0.777133,0.587978,0.829099,4.804230
24,0.6,3.000227,0.0,0.0,-0.671902,-0.190668,8.748954e-17,0.000318,0.00504,0.073089,0.777133,0.576324,0.818168,5.111495
23,0.7,2.881205,0.0,-0.0,-0.633625,-0.083460,8.748954e-17,0.000318,0.00504,0.073089,0.777133,0.562550,0.804669,5.490990


# Ridge

In [188]:
from sklearn.linear_model import Ridge

In [189]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [190]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [191]:
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

train_score: 0.66728240095804
test_score: 0.6462843705034473


,alpha,coef1,coef2,coef3,train_score,test_score,e,coeff_used
0,5.000,4.441198,0.104099,-1.402750,0.665962,0.646284,3.354514,3.0
1,4.999,4.441240,0.104085,-1.402767,0.665962,0.646280,3.354555,3.0
2,4.998,4.441281,0.104072,-1.402785,0.665963,0.646276,3.354595,3.0
3,4.997,4.441323,0.104058,-1.402802,0.665963,0.646272,3.354636,3.0
4,4.996,4.441365,0.104044,-1.402819,0.665964,0.646267,3.354676,3.0
5,4.995,4.441406,0.104030,-1.402837,0.665964,0.646263,3.354717,3.0


In [317]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [318]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'coef1_p', 'coef2_p', 'coef3_p', 'coef4_p', 'coef5_p', 'train_score', 'test_score', 'e'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    p_value = list(f_regression(x_test.values, y_predict)[1])
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + p_value + [score_train, score, e]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [319]:
print('train_score:',max(alpha_df['train_score']))
print('test_score:',max(alpha_df['test_score']))
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

train_score: 0.6585122571984281
test_score: 0.7544180451815767


,alpha,coef1,coef2,coef3,coef4,coef5,coef1_p,coef2_p,coef3_p,coef4_p,coef5_p,train_score,test_score,e
0,5.000,3.884055,0.303819,0.154633,-0.804263,-0.956155,1.981900e-26,0.001636,0.00014,0.000002,0.799893,0.657301,0.754418,6.388850
1,4.999,3.884093,0.303813,0.154644,-0.804262,-0.956169,1.982238e-26,0.001636,0.00014,0.000002,0.799902,0.657302,0.754418,6.388857
2,4.998,3.884131,0.303807,0.154655,-0.804261,-0.956183,1.982576e-26,0.001636,0.00014,0.000002,0.799911,0.657302,0.754418,6.388864
3,4.997,3.884168,0.303801,0.154666,-0.804260,-0.956197,1.982913e-26,0.001636,0.00014,0.000002,0.799920,0.657303,0.754417,6.388870
4,4.996,3.884206,0.303795,0.154676,-0.804259,-0.956211,1.983251e-26,0.001636,0.00014,0.000002,0.799929,0.657303,0.754417,6.388877
5,4.995,3.884243,0.303789,0.154687,-0.804258,-0.956225,1.983589e-26,0.001636,0.00014,0.000002,0.799938,0.657304,0.754417,6.388884


# SVR

In [213]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

In [214]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# sc = StandardScaler()
# sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
# x_train_std = sc.transform(x_train)
# x_test_std = sc.transform(x_test)

In [216]:
c_df = pd.DataFrame(columns=['c', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train.values, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train.values)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.predict(x_test.values)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [217]:
print('c_df:',max(alpha_df['train_score']))
print('c_df:',max(alpha_df['test_score']))
c_df.sort_values(by='test_score', ascending=False)[0:6]

train_score: 0.6586496596708757
test_score: 0.7729431644215765


,c,coef1,coef2,coef3,train_score,test_score,e,coeff_used
98,2.0,1.007111,-0.018663,-0.524920,0.636665,0.770790,5.163847,3.0
96,4.0,1.007122,-0.018882,-0.524921,0.636664,0.770758,5.164566,3.0
11,89.0,1.007159,-0.018628,-0.525132,0.636665,0.770746,5.164828,3.0
97,3.0,1.007063,-0.018701,-0.525030,0.636665,0.770744,5.164880,3.0
5,95.0,1.007154,-0.018502,-0.525209,0.636666,0.770743,5.164888,3.0
81,19.0,1.007155,-0.018622,-0.525176,0.636666,0.770740,5.164973,3.0


In [226]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# sc = StandardScaler()
# sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
# x_train_std = sc.transform(x_train)
# x_test_std = sc.transform(x_test)

In [315]:
c_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'coef1_p', 'coef2_p', 'coef3_p', 'coef4_p', 'coef5_p', 'train_score', 'test_score', 'e'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train.values, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train.values)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.predict(x_test.values)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    p_value = list(f_regression(x_test.values, y_predict)[1])
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0])+ p_value + [score_train, score, e]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [316]:
print('train_score:',max(c_df['train_score']))
print('test_score:',max(c_df['test_score']))
c_df.sort_values(by='test_score', ascending=False)[0:6]

train_score: 0.8021993783469004
test_score: 0.46243317034036435


,alpha,coef1,coef2,coef3,coef4,coef5,coef1_p,coef2_p,coef3_p,coef4_p,coef5_p,train_score,test_score,e
45,55.0,0.986618,-0.025050,-0.038976,-0.436207,-0.256500,1.371124e-22,0.004976,0.000512,0.018438,0.135185,0.802195,0.462433,20.151475
48,52.0,0.986618,-0.025056,-0.038987,-0.436187,-0.256508,1.370820e-22,0.004976,0.000512,0.018439,0.135184,0.802195,0.462432,20.151521
41,59.0,0.986623,-0.025038,-0.038988,-0.436189,-0.256490,1.368084e-22,0.004975,0.000512,0.018443,0.135214,0.802195,0.462430,20.151581
77,23.0,0.986598,-0.025098,-0.039071,-0.435930,-0.256591,1.365216e-22,0.004977,0.000511,0.018458,0.135195,0.802195,0.462419,20.151999
7,93.0,0.986666,-0.025047,-0.038865,-0.435803,-0.256565,1.352063e-22,0.004973,0.000512,0.018483,0.135301,0.802196,0.462419,20.152015
3,97.0,0.986664,-0.025013,-0.038805,-0.435805,-0.256518,1.347018e-22,0.004970,0.000512,0.018491,0.135355,0.802195,0.462418,20.152050


# LinearRegression

In [202]:
from sklearn.linear_model import LinearRegression

In [203]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [204]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score_train, score, e, coeff_used]

coef: [ 4.46592634  0.06357718 -1.35955519]
score_train: 0.6401981679388165
score_prediction: 0.8148616306550289
e: 3.1491292009718395


In [205]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [206]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score, e, coeff_used]

coef: [ 4.18158537  0.25835828  0.3891332  -0.87641394 -0.70706281]
score_train: 0.6742320622269391
score_prediction: 0.7177033747243396
e: 5.904786682514618
